In [1]:
import duckdb
import sqlite3
from typing import List, Dict

In [2]:
duck = duckdb.connect('data/store.duckdb')
sql_con = sqlite3.connect('data/fts.sqlite')

In [ ]:
query = """
WITH
  ocr AS (
    SELECT event_id, video_id, ts, ocr_text
    FROM ocr_event
    WHERE video_id = 'demo1'
      AND ocr_text GLOB '[A-Z][A-Za-z]* [A-Z][A-Za-z]*'
  ),
  intro AS (
    SELECT utt_id, video_id, "start", "end", text
    FROM utterance
    WHERE video_id = 'demo1'
      AND (
        text ILIKE '%my name is %' OR
        text ILIKE '%i am %'       OR
        text ILIKE '%i''m %'       OR
        text ILIKE '%introduc%'
      )
  )
SELECT DISTINCT
       ocr.ocr_text      AS candidate_name,
       ocr.ts            AS visual_ts,
       intro."start"     AS speak_start,
       intro.text        AS spoken_line
FROM ocr
LEFT JOIN intro
  ON ocr.video_id = intro.video_id
 AND ocr.ts BETWEEN intro."start" - 5 AND intro."end" + 5
ORDER BY ocr.ts
LIMIT 10;
"""
df = duck.execute(query).df()
df

,candidate_name,visual_ts,speak_start,spoken_line
0,AI AGENTS,1.333333,NaN,None
1,Master Inventor,2.000000,NaN,None
2,Master Inventor,2.333333,NaN,None
3,Martin Keen,2.666667,NaN,None
4,Master Inventor,2.666667,NaN,None
5,Master Inventor,3.000000,NaN,None
6,Martin Keen,3.000000,NaN,None
7,Master Inventor,3.333333,NaN,None
8,Master Inventor,3.666667,NaN,None
9,Master Inventor,4.000000,NaN,None


: 

In [7]:
query = """
SELECT "start", "end", text
FROM utterance
WHERE video_id = 'demo1'
ORDER BY "start";
"""
df = duck.execute(query).df()

In [8]:
df

,start,end,text
0,0.00,4.44,AI agents can reason and act autonomously to a...
1,4.50,8.13,And unlike a chatbot that only responds one pr...
2,8.31,13.29,AI agents maintain state and can break down co...
3,13.41,16.05,execute them in a sequence or all at once in p...
4,16.26,19.23,and then adjust their plan based on intermedia...
...,...,...,...
120,509.16,514.77,"Supply chain agents use predictive analytics, ..."
121,514.98,520.05,and transportation agents are good examples of...
122,520.11,524.61,continuously recalculating optimal routes as c...
123,524.61,531.54,change. And each of these use cases follows th...


In [13]:
df_person = duck.execute("""
  SELECT video_id,
         ts AS timestamp,
         label,
         conf
  FROM visual_event
  WHERE label ILIKE '%person%'
    AND video_id = 'demo1'
  ORDER BY ts;
""").df()

In [14]:
df_person

,video_id,timestamp,label,conf


In [11]:
df_sample = duck.execute("""
  SELECT video_id,
         ts AS timestamp,
         label,
         conf
  FROM visual_event
  ORDER BY ts
  LIMIT 10;
""").df()

In [12]:
df_sample

,video_id,timestamp,label,conf
0,demo1,0.000000,text,0.741512
1,demo1,0.000000,text,0.994638
2,demo1,0.666667,text,0.788691
3,demo1,0.666667,text,0.993708
4,demo1,1.000000,text,0.890838
5,demo1,1.000000,text,0.993914
6,demo1,1.333333,text,0.874128
7,demo1,1.666667,text,0.994941
8,demo1,1.666667,text,0.951283
9,demo1,2.000000,text,0.891599


In [5]:
cloud_mentions = sql_con.execute("""
    SELECT u.start, u.text
    FROM utterance_fts f
    JOIN utterance u ON u.utt_id = f.rowid
    WHERE f.text MATCH 'cloud'
""").fetchall()

In [6]:
cloud_mentions

[]

In [1]:
import requests
r = requests.post(
    "https://iam.cloud.ibm.com/identity/token",
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type":"urn:ibm:params:oauth:grant-type:apikey", "apikey": "dmZongaiZbrry1YfN1ZpekxBaErUOddXu7dZ8n08Qp_b"}
)
iam_token = r.json()["access_token"]

KeyError: 'access_token'

In [2]:
r.content

b'{"errorCode":"BXNIM0415E","errorMessage":"Provided API key could not be found.","context":{"requestId":"bnc5Z2I-3851cd18029f4cd98358e5b135ef5043","requestType":"incoming.Identity_Token","userAgent":"python-requests/2.31.0","url":"https://iam.cloud.ibm.com","instanceId":"iamid-10-8-5219-d6d71c4-7d77df44bf-nw9gb","threadId":"1f44ee","host":"iamid-10-8-5219-d6d71c4-7d77df44bf-nw9gb","startTime":"02.09.2025 10:40:19:852 UTC","endTime":"02.09.2025 10:40:19:940 UTC","elapsedTime":"88","locale":"en_US","clusterName":"iam-id-prod-eu-de-fra05"}}'

In [2]:
iam_token

'eyJraWQiOiIyMDE5MDcyNCIsImFsZyI6IlJTMjU2In0.eyJpYW1faWQiOiJaNlIwM3VhNmFVLTA0ZmU0ZDNlLWI0NGMtNDQyYS04OWM4LWFiY2NiNWNhNDQ0NCIsImlkIjoiWjZSMDN1YTZhVS0wNGZlNGQzZS1iNDRjLTQ0MmEtODljOC1hYmNjYjVjYTQ0NDQiLCJyZWFsbWlkIjoiWjZSMDN1YTZhVSIsImp0aSI6IjVjYWRmMTAyLTk2MGQtNDQwMy1hN2YxLWY0MWQ0NWUzNjUzNSIsImlkZW50aWZpZXIiOiIwNGZlNGQzZS1iNDRjLTQ0MmEtODljOC1hYmNjYjVjYTQ0NDQiLCJnaXZlbl9uYW1lIjoibm90c2V0IiwiZmFtaWx5X25hbWUiOiJub3RzZXQiLCJuYW1lIjoidXNlcl82OGI2MTgwZTg1QHRlY2h6b25lLmlibS5jb20iLCJlbWFpbCI6InVzZXJfNjhiNjE4MGU4NUB0ZWNoem9uZS5pYm0uY29tIiwic3ViIjoidXNlcl82OGI2MTgwZTg1QHRlY2h6b25lLmlibS5jb20iLCJhdXRobiI6eyJzdWIiOiJ1c2VyXzY4YjYxODBlODVAdGVjaHpvbmUuaWJtLmNvbSIsImlhbV9pZCI6Ilo2UjAzdWE2YVUtMDRmZTRkM2UtYjQ0Yy00NDJhLTg5YzgtYWJjY2I1Y2E0NDQ0IiwibmFtZSI6InVzZXJfNjhiNjE4MGU4NUB0ZWNoem9uZS5pYm0uY29tIiwiZ2l2ZW5fbmFtZSI6Im5vdHNldCIsImZhbWlseV9uYW1lIjoibm90c2V0IiwiZW1haWwiOiJ1c2VyXzY4YjYxODBlODVAdGVjaHpvbmUuaWJtLmNvbSJ9LCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiI4NDcxYWYxODk5YjM0NTY0YjJiMDRiZTc5OWY1MGQ3NSIsImltc19

In [3]:
import requests

url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2023-05-29"

body = {
	"messages": [{"role":"system","content":"You are Mixtral Chat, an AI language model developed by Mistral AI. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."}],
	"project_id": "9917d93b-b2f9-4049-8dd9-d8772fc37cae",
	"model_id": "mistralai/mistral-medium-2505",
	"frequency_penalty": 0,
	"max_tokens": 2000,
	"presence_penalty": 0,
	"temperature": 0,
	"top_p": 1
}

headers = {
	"Accept": "application/json",
	"Content-Type": "application/json",
	"Authorization": "Bearer " + iam_token
}

response = requests.post(
	url,
	headers=headers,
	json=body
)

if response.status_code != 200:
	raise Exception("Non-200 response: " + str(response.text))

data = response.json()

In [4]:
data

{'id': 'chatcmpl-edb2e09d-4694-430c-aeb7-03e092e71508---e46f821081c25f99b97c43b4291d47af---1928bdf0-e144-4ed8-8d7f-13c6d4a4b182',
 'object': 'chat.completion',
 'model_id': 'mistralai/mistral-medium-2505',
 'model': 'mistralai/mistral-medium-2505',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Hello! I'm here to help. How can I assist you today?"},
   'finish_reason': 'stop'}],
 'created': 1756774889,
 'model_version': '1.0.0',
 'created_at': '2025-09-02T01:01:29.790Z',
 'usage': {'completion_tokens': 16, 'prompt_tokens': 45, 'total_tokens': 61},
 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.',
    'id': 'disclaimer_warning',
    'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}